In [1]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import time
import re

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
kor_url_head = 'http://www.newdaily.co.kr/news/search_result.html?search=카이스트&s=&t=date&pn='
eng_url_head = 'http://www.newdaily.co.kr/news/search_result.html?search=kaist&s=&t=date&pn='

# 검색 결과에 따른 url 링크 저장
url1 = []
# url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
page = 1
while page<=107:
    url_final = kor_url_head + str(page)
    try:
        response = requests.get(url_final, headers=headers)
        html = response.content
        soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
        elements = soup.select('#search_result a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
    except:
        print('Request Failed_1')
    for element in elements:
        link = element.attrs['href']
        url1.append(link)
    page += 1

In [ ]:
url1

In [3]:
# 검색 결과에 따른 url2 링크 저장
url2 = []
page = 1
while page<=56:
    url_final = eng_url_head + str(page)
    try:
        response = requests.get(url_final, headers=headers)
        html = response.content
        soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
        elements = soup.select('#search_result a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
    except:
        print('Request Failed_1')
    for element in elements:
        link = element.attrs['href']
        url2.append(link)
    page += 1

In [4]:
url = list(set(url1).union(set(url2)))
df = pd.DataFrame({"url": url}, columns=["url"])
print("Total number of articles: ", len(df))
df.head()

Total number of articles:  1380


,url
0,http://biz.newdaily.co.kr/site/data/html/2015/...
1,http://cc.newdaily.co.kr/site/data/html/2017/1...
2,http://biz.newdaily.co.kr/site/data/html/2015/...
3,http://biz.newdaily.co.kr/site/data/html/2017/...
4,http://cc.newdaily.co.kr/site/data/html/2017/0...


In [ ]:
contents = []
titles = []
urls = []
for row in tqdm(df.itertuples()):
    link = getattr(row, 'url')
    response = requests.get(link, headers=headers)
    html = response.text
    soup = bs(html,'html.parser')
    try:
        for script in soup(["script", "style"]):
            script.decompose() 
        title = soup.select('.nd-news-tit')[0].get_text()
        content = soup.select('.nd-center')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
        content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
        title = title.replace('\xa0', " ")
        title = title.replace('\n', " ")
        titles.append(title)
        contents.append(content)
        urls.append(link)
    except:
        print(row)
df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
df.to_excel("articles/newdaily.xlsx",index = False)
df.to_pickle("pickle files/newdaily.pkl")

1203it [04:55,  4.52it/s]